In [4]:
u = [[1,2],[3,4],[5,6],[7,8],[9,10]]


In [5]:
print(u)

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]


In [13]:
del u[0]

In [14]:
u

[[3, 4], [5, 6], [7, 8], [9, 10]]

In [ ]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    ret = list(set(a) | set(b))
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        
        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}
        search_space = {}

        # iterate through all items in s and create a search space for each item
        for item, support in self.s.items():
            search_space[item] = {}

            # store the itemset as a list to preserve the order of elements
            search_space[item]['itemset'] = [item]

            # store itemset in search space as a python set for easy set operations
            search_space[item]['data'] = set([item])

            # keep track if an itemset has been pruned or not
            search_space[item]['pruned'] = False

            # store the support of the itemset
            search_space[item]['support'] = support

            # create a node in the tree for the item
            tree[item] = {}

        return tree, search_space

    def computeItemsetSupport(self, itemset):
        '''Return support of itemset'''
        count = 0
        for transaction in self.data:
            if set(itemset).issubset(transaction):
                count += 1
        return count

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)
    def prune(self, k, tree, search_space):
        """
        In this method we will find out which itemsets in the current search space are frequent itemsets
        then add them to L[k]. We also prune itemsets that are not frequent itemsets.
        """
        if self.L.get(k) is None:
            self.L[k] = []

        for item, data in search_space.items():
            if data['pruned']:
                continue

            # If an itemset has a support less than the minimum support, prune it.
            if data['support'] < self.minSup:
                search_space[item]['pruned'] = True

            # If an itemset has a support greater than or equal to the minimum support, it is a frequent itemset.
            # Add it to L[k].
            else:
                self.L[k].append(item)

    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
        why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = joinset(search_space[a]['itemset'], search_space[b]['itemset'])
                
                # Second add newset to search_space
                sub_search_space[newset] = {'itemset': newset,
                                            'pruned': False,
                                            'support': self.computeItemsetSupport(newset)}
                # Third, create subtree for newset
                sub_tree[newset] = {}

                for item in newset:
                    if item == a or item == b:
                        continue
                    else:
                        # Add item to newset and check its support
                        tmpset = newset.copy()
                        tmpset.remove(item)
                        tmp_support = self.computeItemsetSupport(tmpset + [item])
                        # If support of item in newset is greater or equal to minSup, then add it to search_space
                        if tmp_support >= self.minSup:
                            sub_search_space[newset][item] = {'itemset': tmpset + [item],
                                                            'pruned': False,
                                                            'support': tmp_support}
                            sub_tree[newset][item] = {}

            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)
    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L